In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls '/content/drive'



MyDrive


In [4]:
train_path = '/content/drive/MyDrive/chaitanya/train'

In [5]:
test_path = '/content/drive/MyDrive/chaitanya/test'

In [116]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [117]:
IMAGE_SIZE = [224, 224]

In [118]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


In [119]:
# don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [120]:
x = Flatten()(vgg16.output)

In [121]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [122]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [123]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [124]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [126]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 2640 images belonging to 2 classes.


In [128]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


Found 660 images belonging to 2 classes.


In [129]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
83/83 [==============================] - 221s 3s/step - loss: 0.4832 - accuracy: 0.7795 - val_loss: 0.4103 - val_accuracy: 0.8288
Epoch 2/5
83/83 [==============================] - 36s 431ms/step - loss: 0.3680 - accuracy: 0.8318 - val_loss: 0.3685 - val_accuracy: 0.8394
Epoch 3/5
83/83 [==============================] - 35s 419ms/step - loss: 0.3378 - accuracy: 0.8447 - val_loss: 0.5034 - val_accuracy: 0.7712
Epoch 4/5
83/83 [==============================] - 35s 424ms/step - loss: 0.3302 - accuracy: 0.8473 - val_loss: 0.3427 - val_accuracy: 0.8470
Epoch 5/5
83/83 [==============================] - 35s 421ms/step - loss: 0.3265 - accuracy: 0.8610 - val_loss: 0.4144 - val_accuracy: 0.8455


In [130]:
labels = {
    0 : "benign" , 
    1 : "Malignant"
}

In [131]:
path = r'/content/drive/MyDrive/chaitanya/test/malignant/1003.jpg'

In [132]:
img = image.load_img(path, target_size=(224, 224))
x = image.img_to_array(img)/255
x = np.expand_dims(x, axis=0)
classes = model.predict([x])

In [133]:
pred_labels = np.argmax(classes, axis = 1)
pred_labels

array([1])

In [135]:
for i in pred_labels:
    print(labels[i])

Malignant
